In [760]:
"""
Скрипт для подсчета общих данных по ЦК
"""

'\nСкрипт для подсчета общих данных по ЦК\n'

In [761]:
import pandas as pd
import numpy as np
import openpyxl
import os
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
warnings.filterwarnings('ignore', category=DeprecationWarning)
import copy
import time

In [762]:
def check_horizont_all_sum_error(df:pd.DataFrame,tup_exluded_cols:tuple,name_itog_cols,name_file):
    """
    Функция для проверки горизонтальных сумм по всей строке
    сумма в колонке 05 должна быть равна сумме всех колонок за исключением 07 и 15
    """
    # датафрейм для ощибок по горизонтали
    hor_error_df = pd.DataFrame(columns=['Название файла', 'Строка или колонка с ошибкой', 'Описание ошибки'])
    
    # получаем список колонок
    all_sum_cols =list(df)
    # удаляем колонки
    for name_cols in tup_exluded_cols:
        all_sum_cols.remove(name_cols)
    # удаляем итоговую колонку
    all_sum_cols.remove(name_itog_cols)

    # получаем сумму колонок за вычетом исключаемых и итоговой колонки
    df['Сумма'] = df[all_sum_cols].sum(axis=1)
    # Проводим проверку
    df['Результат'] = df[name_itog_cols] == df['Сумма']
    df['Результат'] = df['Результат'].apply(lambda x: 'Правильно' if x else 'Неправильно')
    # получаем датафрейм с ошибками и извлекаем индекс
    df = df[df['Результат'] == 'Неправильно'].reset_index()
    # создаем датафрейм дял добавления в ошибки
    temp_error_df = pd.DataFrame(columns=['Название файла', 'Строка или колонка с ошибкой', 'Описание ошибки', ])
    # обрабатываем индексы строк с ошибками чтобы строки совпадали с файлом excel
    raw_lst_index = df['index'].tolist()  # делаем список
    finish_lst_index = list(map(lambda x: x + 1, raw_lst_index))
    finish_lst_index = list(map(lambda x: f'Строка 0{str(x)}', finish_lst_index))
    temp_error_df['Строка или колонка с ошибкой'] = finish_lst_index
    temp_error_df['Название файла'] = name_file
    temp_error_df['Описание ошибки'] = f'Не выполняется условие: гр. {name_itog_cols} = сумма остальных гр. за искл.{tup_exluded_cols} ДАННЫЕ ФАЙЛА НЕ ОБРАБОТАНЫ !!!'
    return temp_error_df
    
    

    
    
    

In [763]:
def check_horizont_chosen_sum_error(df:pd.DataFrame,tup_checked_cols:list,name_itog_cols,name_file):
    """
    Функция для проверки равенства одиночных или небольших групп колонок
    tup_checked_cols колонки сумму которых нужно сравнить с name_itog_cols чтобы она не превышала это значение
    """
    # Считаем проверяемые колонки
    df['Сумма'] = df[tup_checked_cols].sum(axis=1)
     # Проводим проверку
    df['Результат'] = df[name_itog_cols] >= df['Сумма']
    df['Результат'] = df['Результат'].apply(lambda x: 'Правильно' if x else 'Неправильно')
    # получаем датафрейм с ошибками и извлекаем индекс
    df = df[df['Результат'] == 'Неправильно'].reset_index()
    # создаем датафрейм дял добавления в ошибки
    temp_error_df = pd.DataFrame(columns=['Название файла', 'Строка или колонка с ошибкой', 'Описание ошибки', ])
    # обрабатываем индексы строк с ошибками чтобы строки совпадали с файлом excel
    raw_lst_index = df['index'].tolist()  # делаем список
    finish_lst_index = list(map(lambda x: x + 1, raw_lst_index))
    finish_lst_index = list(map(lambda x: f'Строка 0{str(x)}', finish_lst_index))
    temp_error_df['Строка или колонка с ошибкой'] = finish_lst_index
    temp_error_df['Название файла'] = name_file
    temp_error_df['Описание ошибки'] = f'Не выполняется условие: гр. {name_itog_cols} >= сумма {tup_checked_cols} ДАННЫЕ ФАЙЛА НЕ ОБРАБОТАНЫ !!!'
    return temp_error_df
    
    

In [764]:
def check_vertical_chosen_sum(df:pd.DataFrame,lst_checked_rows:list,itog_row,name_file):
    """
    Функция для проверки вертикальной суммы заданных строк сумма значений в tupl_checked_row должна быть равной ил меньше чем значение
    в itog_row
    """
    
    # обрабаотываем список строк чтобы привести его в читаемый вид
    lst_out_rows = list(map(lambda x:x+1,lst_checked_rows))
                        
    # делаем значения строковыми
    lst_out_rows = list(map(str,lst_out_rows))
    # Добавляем ноль в строки
    lst_out_rows = list(map(lambda x:'0'+x,lst_out_rows))
    #обрабатываем формат выходной строки
    out_itog_row = f'0{str(itog_row+1)}'
    
    # создаем временный датафрейм
    foo_df = pd.DataFrame()
    # разворачиваем строки в колонки
    for idx_row in lst_checked_rows:
        foo_df[idx_row] = df.iloc[idx_row,:]
    
    # добавляем итоговую колонку
    foo_df[itog_row] = df.iloc[itog_row,:]
    
    # суммируем
    foo_df['Сумма'] = foo_df[lst_checked_rows].sum(axis=1)
    foo_df['Результат'] = foo_df[itog_row] >= foo_df['Сумма']
    foo_df['Результат'] = foo_df['Результат'].apply(lambda x: 'Правильно' if x else 'Неправильно')
        # получаем датафрейм с ошибками и извлекаем индекс
    error_df = foo_df[foo_df['Результат'] == 'Неправильно'].reset_index()
    # Добавляем слово колонка
    error_df['index'] = error_df['index'].apply(lambda x:'Колонка '+str(x))
    # создаем датафрейм дял добавления в ошибки
    temp_error_df = pd.DataFrame(columns=['Название файла', 'Строка или колонка с ошибкой', 'Описание ошибки', ])    
    temp_error_df['Строка или колонка с ошибкой'] = error_df['index']    
    temp_error_df['Описание ошибки'] = f'Для указанной колонки сумма в строках {lst_out_rows} превышает значением в строке {out_itog_row} ДАННЫЕ ФАЙЛА НЕ ОБРАБОТАНЫ !!! '
    temp_error_df['Название файла'] =name_file
    
    return temp_error_df

    
    
    
    

In [765]:
def check_error_ck(df:pd.DataFrame,name_file):
        # создаем датафрейм для регистрации ошибок
    error_df = pd.DataFrame(columns=['Название файла', 'Строка или колонка с ошибкой', 'Описание ошибки'])
    
    # проводим горизонтальные проверки
    # проверка на общую сумму
    first_error_ck_df  = check_horizont_all_sum_error(df.copy(),('07','15'),'05',name_file) 
    error_df = pd.concat([error_df,first_error_ck_df],axis=0,ignore_index=True)
    # проверяем небольшие группы или одиночные колонки
    second_error_ck_df  = check_horizont_chosen_sum_error(df.copy(),['07'],'06',name_file) 
    error_df = pd.concat([error_df,second_error_ck_df],axis=0,ignore_index=True)
    
    #проверяем колонки 14 и 15
    third_error_ck_df  = check_horizont_chosen_sum_error(df.copy(),['15'],'14',name_file) 
    error_df = pd.concat([error_df,third_error_ck_df],axis=0,ignore_index=True)
    
    # Проводим вертикальные проверки
    # Сумма овз и целевиков не должна превышать общую численность выпускников. Строки с индексом 1 и 4 должныть меньше или равны строке с индексом 0
    fourth_error_ck_df = check_vertical_chosen_sum(df.copy(),[1,4],0,name_file)
    error_df = pd.concat([error_df,fourth_error_ck_df],axis=0,ignore_index=True)
    
    #Проверяем ОВЗ
    fifth_error_ck_df = check_vertical_chosen_sum(df.copy(),[2,3],1,name_file)
    error_df = pd.concat([error_df,fifth_error_ck_df],axis=0,ignore_index=True)
    
    
    return error_df

 
        
    
    


In [ ]:
path_folder_data_ck = 'data/Отчет ЦК'
path_to_end_folder_ck = 'data'

In [766]:
# создаем базовый датафрейм заполненный нулями
base_df = pd.DataFrame(np.zeros((5,27)))
base_df=base_df.applymap(int) # приводим его к инту
cols_df = ['05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24',
                   '25','26','27','28','29','30','31']
base_df.columns = cols_df

# Создаем общую таблицы для проверки
general_table= pd.DataFrame(columns=['Название файла']+cols_df)

# создаем датафрейм для регистрации ошибок
base_error_df = pd.DataFrame(columns=['Название файла', 'Строка или колонка с ошибкой', 'Описание ошибки', ])

#Создаем датафрейм для хранения строковых данных из колонки 32
str_df = pd.DataFrame(index=range(5))

for file in os.listdir(path_folder_data_ck):
    if not file.startswith('~$') and file.endswith('.xlsx'):
        name_file = file.split('.xlsx')[0]
        temp_df_ck = pd.read_excel(f'{path_folder_data_ck}/{file}',skiprows=5,nrows=5)
        if temp_df_ck.shape[1] !=30:
            temp_error_df = pd.DataFrame(data=[
                    [f'{name_file}', '', 'Количество колонок в таблице не равно 30 !!! ДАННЫЕ ФАЙЛА НЕ ОБРАБОТАНЫ !!!']],
                                             columns=['Название файла', 'Строка или колонка с ошибкой',
                                                      'Описание ошибки'])
            base_error_df = pd.concat([base_error_df, temp_error_df], axis=0, ignore_index=True)
            continue
        # Создаем копию датафрейма только с числовыми колонками
        temp_df_int = temp_df_ck.iloc[:,2:29].copy()
        # заменяем пустые ячейки нулями

        temp_df_int.fillna(0,inplace=True)
        temp_df_int = temp_df_int.applymap(int)
        
        

        # проверяем на ошибки
        temp_error_df = check_error_ck(temp_df_int.copy(),name_file)
        # Добавляем в итоговый датафрейм с ошибками
        base_error_df = pd.concat([base_error_df,temp_error_df],axis=0,ignore_index=True)
        # проверяем размер датафрейма с ошибками, если их нет то добавляем в результат.
        if base_error_df.shape[0] == 0:
            base_df = base_df + temp_df_int # складываем значения в таблицах
            # делаем копию промежутчного датафрейма, так как мы будем добавлять новую колонку
            temp_add_df = temp_df_int.copy()
            temp_add_df.insert(0,'Название файла',name_file)
            temp_add_df['32'] = temp_df_ck.iloc[:,29]
            general_table = pd.concat([general_table,temp_add_df],axis=0,ignore_index=True) # сохраняем в общую таблицу
            # Добаввляем принимаемые меры
            str_df = pd.concat([str_df,temp_df_ck.iloc[:,29].to_frame().fillna('_')],axis=1,ignore_index=True)            
            
        else:
            continue
        
        



In [768]:
# Объдиняем колонки с принимаемыми мерами в одну и добавляем в base df

base_df['32'] = str_df.apply(lambda x:';'.join(x),axis=1)
# Добавляем колонки
fourth = ['Всего (общая численность выпускников)','из общей численности выпускников (из строки 01): лица с ОВЗ',
          'из числа лиц с ОВЗ (из строки 02): инвалиды и дети-инвалиды','Инвалиды и дети-инвалиды (кроме учтенных в строке 03)',
          'Имеют договор о целевом обучении']
three = ['01','02','03','04','05']
base_df.insert(0,'03',three)
base_df.insert(1,'04',fourth)
# в общую таблицу
miultipler = general_table.shape[0] // 5
general_table.insert(1,'03',three*miultipler)
general_table.insert(2,'04',fourth*miultipler)



t = time.localtime()
current_time = time.strftime('%H_%M_%S', t)
base_df.to_excel(f'{path_to_end_folder_ck}/Отчет для ЦК Общий результат от {current_time}.xlsx',index=False)
base_error_df.to_excel(f'{path_to_end_folder_ck}/Отчет для ЦК Ошибки от {current_time}.xlsx',index=False)
general_table.to_excel(f'{path_to_end_folder_ck}/Отчет для ЦК Данные из всех таблиц от {current_time}.xlsx',index=False)

In [769]:
base_df.shape

(5, 30)